# 单例模式

- 通常情况下，我们定义一个类，这个类可以实例化很多个不同的对象，这些对象互不相同

In [6]:
class Person:
    def __init__(self, name):
        self.name = name

p1 = Person('zyp')
p2 = Person("zyp")

print(p1.name, p2.name)
print(p1)
print(p2)
print(p1 is p2)

zyp zyp
False


- 通过上个例子可以看出，尽管实例化时传入的参数一样时，实例化后的对象还是不一样的。
- 如果我们想让这些实例化出来的对象要一样，这个时候就需要使用设计模式中的--**单例设计模式**

## 单例设计模式

- 目的 —— 让 类 创建的对象，在系统中 只有 唯一的一个实例
- 每一次执行 类名() 返回的对象，内存地址是相同的
- Python中实现单例模式需要借助`__new__`方法，所以我们要先了解`__new__`方法

## `__new__`方法

- 使用 类名() 创建对象时，Python 的解释器首先会调用 `__new__` 方法为对象**分配空间**
- `__new__` 是一个 由 object 基类提供的**内置的静态方法**，主要作用有两个：
    1. 在内存中为对象 分配空间
    2. 返回 对象的引用
- `__init__`方法的主要作用有两个
    1. 对象初始化
    2. 定义实例属性
- `__new__和__init__`两个方法共同实现了像c++中的构造方法的作用。
- Python的解释器获得对象的引用后，将引用作为第一个参数，传递给 `__init__` 方法

- **重写 __new__ 方法 的代码非常固定！**
    - 重写 `__new__ 方法 一定要 return super().__new__(cls)`
    - 否则 Python 的解释器 得不到 分配了空间的 对象引用，就不会调用对象的初始化方法
    - 注意：`__new__` 是一个静态方法，在调用时需要 主动传递 cls 参数

In [11]:
class Player(object):
    def __new__(cls, *args, **kwargs):
        print("__new__方法被调用，为对象分配内存空间")
        
        return super().__new__(cls)
    
    def __init__(self):
        print("__init__实例化对象")
        
player = Player()


# __new__方法不返回内容，
class Error(object):
    def __new__(cls, *args, **kwargs):
        print("__new__方法被调用")
        
    def __init__(self):
        print("__init__方法被调用")

e = Error()

__new__方法被调用，为对象分配内存空间
__init__实例化对象
__new__方法被调用


## Python 中的单例

- 单例 —— 让 类 创建的对象，在系统中 只有 唯一的一个实例
- 定义一个 类属性，初始值是 None，用于记录 单例对象的引用
- 重写 `__new__` 方法
- 如果 类属性 is None，调用父类方法分配空间，并在类属性中记录结果
- 返回 类属性 中记录的 对象引用

- ![原理图](https://i.screenshot.net/70j3ji2)

In [19]:
# 1. 判断类属性是否为None
# 2. 调用父类方法，为第一个对象分配内存空间
# 3. 返回类属性保存的对象引用

class Player(object):
    instance = None
    
    def __new__(cls, *args, **kwargs):
        if cls.instance is None:
            cls.instance = super().__new__(cls)
        return cls.instance


player1 = Player()
player2 = Player()

print(player1)
print(player2)


- 上面的例子仅修改`__new__`方法，但是在每次在每次使用 类名() 创建对象时，Python 的解释器都会自动调用两个方法：
    - `__new__` 分配空间
    - `__init__` 对象初始化
- 对`__new__` 方法改造之后，每次都会得到 第一次被创建对象的引用；但是：初始化方法还会被再次调用

In [21]:
# 初始化方法被调用了两次
# 第二次调用的初始化方法会修改第一次初始化的值，因为他们的内存地址是一样的
class Player(object):
    instance = None
    
    def __new__(cls, *args, **kwargs):
        if cls.instance is None:
            cls.instance = super().__new__(cls)
        return cls.instance
    
    def __init__(self, name):
        self.name = name
        
p1 = Player('111')
print(p1.name)

p2 = Player('222')
print(p1.name)
print(p2.name)

111
222
222


## 只执行一次初始化工作的解决办法

1. 定义一个类属性 init_flag 标记是否 执行过初始化动作，初始值为 False
2. 在 `__init__` 方法中，判断 init_flag，如果为 False 就执行初始化动作
3. 然后将 init_flag 设置为 True
4. 这样，再次 自动 调用 `__init__` 方法时，初始化动作就不会被再次执行 了

In [26]:
class Player(object):
    instance = None
    init_flag = False
    
    def __new__(cls, *args, **kwargs):
        if cls.instance is None:
            cls.instance = super().__new__(cls)
        return cls.instance
    
    def __init__(self, name):
        # 如果已经初始化过，则直接返回
        # 没有初始化则将进行初始化，并将初始化标记置为True
        if self.__class__.init_flag == True:
            return
        else:
            self.name = name
            self.__class__.init_flag = True
        
        
# 初始化方法只调用一次
# 只记录第一次初始化的值
p1 = Player('111')
print(p1.name)

p2 = Player('222')
print(p1.name)
print(p2.name)

111
111
111



## 单例设计模式的应用场景

- 音乐播放 对象
- 回收站 对象
- 打印机 对象